In [1]:
import configparser
import os

import openai
import requests
from PIL import Image

config = configparser.ConfigParser()
config.read("config.ini")


engine = ""


def config_model(section="wiki.gpt35"):
    openai.api_key = config[section]["api_key"]
    openai.api_type = config[section]["api_type"]
    openai.api_base = config[section]["api_base"]
    openai.api_version = config[section]["api_version"]
    global engine
    engine = config[section]["engine"]


config_model()  # default gpt4

In [ ]:
# general situation
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Does Azure OpenAI support customer managed keys?"},
    {
        "role": "assistant",
        "content": "Yes, customer managed keys are supported by Azure OpenAI.",
    },
    {"role": "user", "content": "Do other Azure AI services support this too?"},
]

In [11]:
# based on instructions
messages = [
    {
        "role": "system",
        "content": "Assistant is an intelligent chatbot designed to help users answer their tax related questions."
        + "Instructions:"
        + "- Only answer questions related to taxes."
        + '- If you\'re unsure of an answer, you can say "I don\'t know" or "I\'m not sure" and recommend users go to the IRS website for more information.',
    },
    {"role": "user", "content": "How's the weather this weekend?"},
]

In [15]:
# Using data grounding
messages = [
    {
        "role": "system",
        "content": "Assistant is an intelligent chatbot designed to help users answer technical questions about Azure OpenAI Serivce. Only answer questions using the context below and if you're not sure of an answer, you can say 'I don't know'"
        + "Context:"
        + "- Azure OpenAI Service provides REST API access to OpenAI's powerful language models including the GPT-3, Codex and Embeddings model series."
        + "- AI Service gives customers advanced language AI with OpenAI GPT-3, Codex, and DALL-E models with the security and enterprise promise of Azure. Azure OpenAI co-develops the APIs with OpenAI, ensuring compatibility and a smooth transition from one to the other."
        + "- At Microsoft, we're committed to the advancement of AI driven by principles that put people first. Microsoft has made significant investments to help guard against abuse and unintended harm, which includes requiring applicants to show well-defined use cases, incorporating Microsoft’s principles for responsible AI use.",
    },
    {"role": "user", "content": "Which models can I choose from azure openai service?"},
]

In [ ]:
# few shot learning with chat
messages = [
    {
        "role": "system",
        "content": "Assistant is an intelligent chatbot designed to help users answer their tax related questions. ",
    },
    {"role": "user", "content": "When do I need to file my taxes by?"},
    {
        "role": "assistant",
        "content": "In 2023, you will need to file your taxes by April 18th. The date falls after the usual April 15th deadline because April 15th falls on a Saturday in 2023. For more details, see https://www.irs.gov/filing/individuals/when-to-file.",
    },
    {"role": "user", "content": "How can I check the status of my tax refund?"},
    {
        "role": "assistant",
        "content": "You can check the status of your tax refund by visiting https://www.irs.gov/refunds",
    },
]

In [19]:
# use chat completion for non-chat scenario
messages = [
    {
        "role": "system",
        "content": "You are an assistant designed to extract entities from text. Users will paste in a string of text and you will respond with entities you've extracted from the text as a JSON object. Here's an example of your output format:"
        + "{"
        + '"name": "",'
        + '"company": "",'
        + '"phone_number": ""'
        + "}",
    },
    {
        "role": "user",
        "content": "Hello. My name is Robert Smith. I'm calling from Contoso Insurance, Delaware. My colleague mentioned that you are interested in learning about our comprehensive benefits policy. Could you give me a call back at (555) 346-9322 when you get a chance so we can go over the benefits?",
    },
]

In [2]:
# use chat completion for non-chat scenario
messages = [
    {
        "role": "system",
        "content": 'You are an NLP processor that helps users check whether ideas are similar. Please identify the ideas from the idea table that are similar to the user\'s idea and explain why they are similar.\nThe idea table has two columns, one for the UUID of the idea and the other for the content of the idea.\nTransform the result table to JSON format. it contains three keys, first for the UUID of the idea, the second for the similarity score, the third for the explanation.\nFormat of results like:\n{\n"similar_ideas": [\n{\n"UUID": "",\n"similarity_score":"",\n"explanation":""\n}\n]\n}',
    },
    {
        "role": "user",
        "content": "Please find similar ideas to the original one from the idea table.  \n---  \noriginal idea:   \n---  \nidea table:  \nUUID|idea  \n-|-  \n5271F0EB-73D6-8AB1-9853-E203A443562C|It's gonna be a nice day tomorrow.  \n9CD5862D-C3E4-2D01-0273-13D992D85328|I love lunch today.  \n9AEAF8D1-85A8-A23E-02B7-AAEDF7D022D6|Where are we going this weekend?",
    },
]

In [3]:
# general situation
response = openai.ChatCompletion.create(
    engine="openai-api", messages=messages  # engine = "deployment_name".
)
print(response)
print(response["choices"][0]["message"]["content"])

{
  "id": "chatcmpl-8AxHl6xkqq6sraNGIdoEGbpfwUghS",
  "object": "chat.completion",
  "created": 1697621865,
  "model": "gpt-35-turbo",
  "prompt_filter_results": [
    {
      "prompt_index": 0,
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      }
    }
  ],
  "choices": [
    {
      "index": 0,
      "finish_reason": "stop",
      "message": {
        "role": "assistant",
        "content": "{\n\"similar_ideas\": [\n{\n\"UUID\": \"5271F0EB-73D6-8AB1-9853-E203A443562C\",\n\"similarity_score\": \"1.0\",\n\"explanation\": \"The content of the idea 'It's gonna be a nice day tomorrow.' is an exact match to the original idea.\"\n},\n{\n\"UUID\": \"9

In [3]:
# conversation loop
# 注意token的最大值, 持续循环的话token会溢出

conversation = [{"role": "system", "content": "You are a helpful assistant."}]

while True:
    user_input = input()
    conversation.append({"role": "user", "content": user_input})

    response = openai.ChatCompletion.create(engine="openai-api", messages=conversation)

    conversation.append(
        {"role": "assistant", "content": response["choices"][0]["message"]["content"]}
    )
    print("\n" + response["choices"][0]["message"]["content"] + "\n")

 How's the weather this weekend?



I'm sorry, but as an AI language model, I don't have access to real-time weather information. Can I help you with anything else?



 How's the weather in Shanghai today?



I'm sorry, but I don't have access to real-time weather information. However, you can check the weather updates for Shanghai on various weather websites or mobile apps available on the internet.



 What is the resident population of Shanghai



According to the latest data from the United Nations, the resident population of Shanghai as of 2021 is approximately 24.9 million.



 What was China's GDP last year


KeyboardInterrupt: 

In [1]:
# 管理和计算token
import tiktoken

system_message = {"role": "system", "content": "You are a helpful assistant."}
max_response_tokens = 250
token_limit = 4096
conversation = []
conversation.append(system_message)


def num_tokens_from_messages(messages):
    # model to encoding mapping https://github.com/openai/tiktoken/blob/main/tiktoken/model.py
    encoding = tiktoken.get_encoding("cl100k_base")
    num_tokens = 0
    for message in messages:
        # every message follows <im_start>{role/name}\n{content}<im_end>\n
        num_tokens += 4
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":  # if there's a name, the role is omitted
                num_tokens += -1  # role is always required and always 1 token
    num_tokens += 2  # every reply is primed with <im_start>assistant
    return num_tokens


while True:
    user_input = input("")
    conversation.append({"role": "user", "content": user_input})
    conv_history_tokens = num_tokens_from_messages(conversation)

    while conv_history_tokens + max_response_tokens >= token_limit:
        del conversation[1]
        conv_history_tokens = num_tokens_from_messages(conversation)

    response = openai.ChatCompletion.create(
        engine="openai-api",  # The deployment name you chose when you deployed the GPT-35-Turbo or GPT-4 model.
        messages=conversation,
        temperature=0.7,
        max_tokens=max_response_tokens,
    )

    conversation.append(
        {"role": "assistant", "content": response["choices"][0]["message"]["content"]}
    )
    print("\n" + response["choices"][0]["message"]["content"] + "\n")